In [ ]:
import sympy as smp
import numpy as np

# Define symbols
x, y, z, t, I, mu0 = smp.symbols('x y z t I mu_0', real=True)
R, H = smp.symbols('R H', real=True)

# Define parametric functions and vectors
f, g, h = smp.symbols('f g h', cls=smp.Function)
f = f(t)
g = g(t)
h = h(t)

r = smp.Matrix([x, y, z])        # Observation point
l = smp.Matrix([f, g, h])        # Parametric position of the current element
dldt = smp.diff(l, t)            # Derivative of l with respect to t

# Debugging: Print intermediate results
print("Parametric position of the current element (l):", l)
print("Derivative of l with respect to t (dldt):", dldt)

# Calculate the magnetic field contribution (dB/dt)
try:
    dBdt = I * dldt.cross(r - l) / (r - l).norm()**3
    print("dB/dt before substitution:", dBdt)
except Exception as e:
    print("Error calculating dB/dt:", e)
    dBdt = None

# Substitute the parametric equations and specific coordinates
if dBdt is not None:
    try:
        dBdt = dBdt.subs([
            (f, R * smp.cos(t)),
            (g, R * smp.sin(t)),
            (h, 0),
            (x, 0),
            (y, 0),
            (z, H)
        ]).doit()
        print("dB/dt after substitution:", dBdt)
    except Exception as e:
        print("Error during substitution:", e)
        dBdt = None

# Directly attempt symbolic integration
if dBdt is not None:
    try:
        magnetic_field = (mu0 / (4 * smp.pi)) * smp.integrate(dBdt, (t, 0, 2 * smp.pi))
        print("Symbolic Magnetic Field (B):")
        print(magnetic_field)
    except Exception as e:
        print("Symbolic integration failed. Error:", e)

        # Fallback to numerical integration
        print("Attempting numerical integration...")
        try:
            dBdt_numeric = smp.lambdify(t, dBdt, "numpy")
            t_vals = np.linspace(0, 2 * np.pi, 1000)
            dBdt_vals = np.array([dBdt_numeric(ti) for ti in t_vals])
            magnetic_field_numeric = (mu0 / (4 * np.pi)) * np.trapz(dBdt_vals, t_vals)
            print("Numerical Magnetic Field (B):", magnetic_field_numeric)
        except Exception as e_num:
            print("Numerical integration failed. Error:", e_num)
else:
    print("dB/dt is None. Cannot proceed with integration.")



Parametric position of the current element (l): Matrix([[f(t)], [g(t)], [h(t)]])
Derivative of l with respect to t (dldt): Matrix([[Derivative(f(t), t)], [Derivative(g(t), t)], [Derivative(h(t), t)]])
dB/dt before substitution: Matrix([[I*(-(y - g(t))*Derivative(h(t), t) + (z - h(t))*Derivative(g(t), t))/(Abs(x - f(t))**2 + Abs(y - g(t))**2 + Abs(z - h(t))**2)**(3/2)], [I*((x - f(t))*Derivative(h(t), t) - (z - h(t))*Derivative(f(t), t))/(Abs(x - f(t))**2 + Abs(y - g(t))**2 + Abs(z - h(t))**2)**(3/2)], [I*(-(x - f(t))*Derivative(g(t), t) + (y - g(t))*Derivative(f(t), t))/(Abs(x - f(t))**2 + Abs(y - g(t))**2 + Abs(z - h(t))**2)**(3/2)]])
dB/dt after substitution: Matrix([[H*I*R*cos(t)/(H**2 + R**2*sin(t)**2 + R**2*cos(t)**2)**(3/2)], [H*I*R*sin(t)/(H**2 + R**2*sin(t)**2 + R**2*cos(t)**2)**(3/2)], [I*(R**2*sin(t)**2 + R**2*cos(t)**2)/(H**2 + R**2*sin(t)**2 + R**2*cos(t)**2)**(3/2)]])
